# Parkinson's Disease Feature-Centric Comparative Study
## Results Analysis Notebook

This notebook provides interactive exploration of the experiment results from the 2×2 cross-paradigm testing framework.

**Components:**
- Feature Selection: Classical (Random Forest) vs Quantum-Inspired (QIGA)
- Models: XGBoost, MLP (Classical) vs QNN (Quantum-Inspired)

---

In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Results directories
RESULTS_DIR = Path('../results')
METRICS_DIR = RESULTS_DIR / 'metrics'
FIGURES_DIR = RESULTS_DIR / 'figures'
FEATURES_DIR = RESULTS_DIR / 'selected_features'

print("Directories:")
print(f"  Results: {RESULTS_DIR.absolute()}")
print(f"  Metrics: {METRICS_DIR.absolute()}")

Directories:
  Results: C:\Users\saiku\OneDrive\Desktop\SDP_Test\parkinson_feature_study\notebooks\..\results
  Metrics: C:\Users\saiku\OneDrive\Desktop\SDP_Test\parkinson_feature_study\notebooks\..\results\metrics


## 1. Load Results

In [2]:
# Load cross-validation results
try:
    cv_results = pd.read_csv(METRICS_DIR / 'cross_validation_results.csv')
    agg_results = pd.read_csv(METRICS_DIR / 'aggregated_results.csv')
    stats_tests = pd.read_csv(METRICS_DIR / 'statistical_tests.csv')
    stability = pd.read_csv(METRICS_DIR / 'feature_stability.csv')
    
    print(f"Loaded {len(cv_results)} CV results")
    print(f"Loaded {len(agg_results)} aggregated results")
    print(f"Loaded {len(stats_tests)} statistical tests")
except FileNotFoundError as e:
    print(f"Results not found. Run experiments first: python experiments/run_experiment.py")
    cv_results = None

Results not found. Run experiments first: python experiments/run_experiment.py


## 2. Performance Overview

In [3]:
if cv_results is not None:
    # Display aggregated results
    display_cols = ['feature_method', 'model', 'accuracy_mean', 'accuracy_std', 
                   'f1_mean', 'f1_std', 'roc_auc_mean', 'roc_auc_std']
    
    print("\nAggregated Performance (Mean ± Std):")
    print("=" * 80)
    display(agg_results[display_cols].round(4))

In [4]:
if cv_results is not None:
    # Create performance comparison heatmaps
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for ax, fs_method in zip(axes, ['classical', 'quantum_inspired']):
        data = agg_results[agg_results['feature_method'] == fs_method]
        metrics = ['accuracy_mean', 'precision_mean', 'recall_mean', 'f1_mean', 'roc_auc_mean']
        
        heatmap_data = data[['model'] + metrics].set_index('model')
        heatmap_data.columns = [m.replace('_mean', '') for m in metrics]
        
        sns.heatmap(heatmap_data, annot=True, fmt='.3f', cmap='RdYlGn', 
                   vmin=0.5, vmax=1.0, ax=ax)
        ax.set_title(f'{fs_method.replace("_", " ").title()} Features')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'performance_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()

## 3. Statistical Significance

In [5]:
if cv_results is not None:
    print("\nStatistical Tests Summary:")
    print("=" * 80)
    
    # Significant results
    significant = stats_tests[stats_tests['significant'] == True]
    if len(significant) > 0:
        print(f"\n{len(significant)} significant differences found (p < 0.05):")
        display(significant[['comparison', 'metric', 'difference', 'p_value']])
    else:
        print("\nNo statistically significant differences found.")

## 4. Feature Stability Analysis

In [6]:
if cv_results is not None:
    print("\nFeature Selection Stability:")
    print("=" * 80)
    display(stability)
    
    # Visualize stability
    fig, ax = plt.subplots(figsize=(8, 4))
    stability.plot(x='method', y='jaccard_mean', kind='bar', 
                  yerr='jaccard_std', ax=ax, legend=False)
    ax.set_ylabel('Jaccard Similarity')
    ax.set_xlabel('Feature Selection Method')
    ax.set_title('Feature Stability Across Folds')
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'feature_stability.png', dpi=150, bbox_inches='tight')
    plt.show()

## 5. Box Plots by Fold

In [7]:
if cv_results is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for ax, metric in zip(axes, ['accuracy', 'roc_auc']):
        sns.boxplot(
            data=cv_results, 
            x='model', 
            y=metric, 
            hue='feature_method',
            ax=ax
        )
        ax.set_title(f'{metric.replace("_", " ").upper()} by Model and Feature Method')
        ax.set_ylim(0, 1.1)
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'boxplots.png', dpi=150, bbox_inches='tight')
    plt.show()

## 6. Feature Overlap Analysis

In [8]:
# Load and analyze selected features
if FEATURES_DIR.exists():
    classical_features = []
    quantum_features = []
    
    for f in FEATURES_DIR.glob('classical_features_fold_*.csv'):
        df = pd.read_csv(f)
        classical_features.extend(df['feature_name'].tolist())
    
    for f in FEATURES_DIR.glob('quantum_inspired_features_fold_*.csv'):
        df = pd.read_csv(f)
        quantum_features.extend(df['feature_name'].tolist())
    
    if classical_features and quantum_features:
        classical_set = set(classical_features)
        quantum_set = set(quantum_features)
        
        overlap = classical_set & quantum_set
        only_classical = classical_set - quantum_set
        only_quantum = quantum_set - classical_set
        
        print(f"\nFeature Overlap Analysis:")
        print(f"  Features unique to Classical: {len(only_classical)}")
        print(f"  Features unique to Quantum: {len(only_quantum)}")
        print(f"  Overlapping features: {len(overlap)}")
        
        if overlap:
            print(f"\nTop overlapping features: {list(overlap)[:10]}")

## 7. Conclusions

### Key Findings:
1. **Feature Selection Impact**: Compare classical vs quantum-inspired feature selection
2. **Model Performance**: Compare XGBoost, MLP, and QNN across feature sets
3. **Statistical Significance**: Identify meaningful differences
4. **Feature Stability**: Assess consistency of selected features

### Important Notes:
- Quantum-inspired methods are classical simulations, not quantum computing
- Risk scores represent likelihood, NOT medical diagnosis
- Results should be validated on independent datasets before clinical use